<a href="https://colab.research.google.com/github/ai-nlp-gl/ticket-assignment/blob/Thamizh/thamizharasan_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Importing Libraries
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

In [0]:
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
#Reading CSV file
df=pd.read_csv('input_data.csv')  

In [0]:
# Viewing top 10 entries of data set
df.head(10)

,Short description,Description,Caller,Assignment group
0,login issue,-verified user details.(employee# & manager name)\r\n-checked the user name in ad and reset the password.\r\n-advised the user to login and check.\r\n-caller confirmed that he was able to login.\r...,spxjnwir pjlcoqds,GRP_0
1,outlook,"\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail.com\r\n\r\nhello team,\r\n\r\nmy meetings/skype meetings etc are not appearing in my outlook calendar, can somebody please advise how to correct this...",hmjdrvpb komuaywn,GRP_0
2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail.com\r\n\r\nhi\r\n\r\ni cannot log on to vpn\r\n\r\nbest,eylqgodm ybqkwiam,GRP_0
3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0
5,unable to log in to engineering tool and skype,unable to log in to engineering tool and skype,eflahbxn ltdgrvkz,GRP_0
6,event: critical:HostName_221.company.com the value of mountpoint threshold for /oracle/SID_37/erpdata21/,"event: critical:HostName_221.company.com the value of mountpoint threshold for /oracle/SID_37/erpdata21/sr3psa1d_7/sr3psa1d.data7,perpsr3psa1d,4524 is 98",jyoqwxhz clhxsoqy,GRP_1
7,ticket_no1550391- employment status - new non-employee [enter user's name],ticket_no1550391- employment status - new non-employee [enter user's name],eqzibjhw ymebpoih,GRP_0
8,unable to disable add ins on outlook,unable to disable add ins on outlook,mdbegvct dbvichlg,GRP_0
9,ticket update on inplant_874773,ticket update on inplant_874773,fumkcsji sarmtlhy,GRP_0


In [0]:
# Depth Vs Breadth analysis 
df.shape

(8500, 4)

In [0]:
# Understanding the column data with respect to count, uniqueness, and frequency
df.describe()

,Short description,Description,Caller,Assignment group
count,8498,8499,8500,8500
unique,7482,7817,2950,74
top,password reset,the,bpctwhsn kzqsbmtp,GRP_0
freq,38,56,810,3976


In [0]:
 # Understanding the data types of each column & coressponding null entries
df.info()
# 2 null entries in Short description column and 1 null entry Description column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8500 entries, 0 to 8499
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Short description  8498 non-null   object
 1   Description        8499 non-null   object
 2   Caller             8500 non-null   object
 3   Assignment group   8500 non-null   object
dtypes: object(4)
memory usage: 265.8+ KB


In [0]:
# Dropping the caller ID column as ticket classification depends only on words present in short and main description column
df1 = df.drop(['Caller'], axis = 1) 

In [0]:
# Viewing the dataset post dropping the caller column
df1.head(10)

,Short description,Description,Assignment group
0,login issue,-verified user details.(employee# & manager name)\r\n-checked the user name in ad and reset the password.\r\n-advised the user to login and check.\r\n-caller confirmed that he was able to login.\r...,GRP_0
1,outlook,"\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail.com\r\n\r\nhello team,\r\n\r\nmy meetings/skype meetings etc are not appearing in my outlook calendar, can somebody please advise how to correct this...",GRP_0
2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail.com\r\n\r\nhi\r\n\r\ni cannot log on to vpn\r\n\r\nbest,GRP_0
3,unable to access hr_tool page,unable to access hr_tool page,GRP_0
4,skype error,skype error,GRP_0
5,unable to log in to engineering tool and skype,unable to log in to engineering tool and skype,GRP_0
6,event: critical:HostName_221.company.com the value of mountpoint threshold for /oracle/SID_37/erpdata21/,"event: critical:HostName_221.company.com the value of mountpoint threshold for /oracle/SID_37/erpdata21/sr3psa1d_7/sr3psa1d.data7,perpsr3psa1d,4524 is 98",GRP_1
7,ticket_no1550391- employment status - new non-employee [enter user's name],ticket_no1550391- employment status - new non-employee [enter user's name],GRP_0
8,unable to disable add ins on outlook,unable to disable add ins on outlook,GRP_0
9,ticket update on inplant_874773,ticket update on inplant_874773,GRP_0


In [0]:
# Creating a replace function to remove the letters GRP_ from the dependent variable column
def myreplace(s):
    s = s.replace('GRP_', '')
    return s

In [0]:
# Calling the function 
df1['Assignment group'] = df1['Assignment group'].map(myreplace)

In [0]:
# Viewing the dataset post replacing dependent variable data points
df1

,Short description,Description,Assignment group
0,login issue,-verified user details.(employee# & manager na...,0
1,outlook,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...,0
2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...,0
3,unable to access hr_tool page,unable to access hr_tool page,0
4,skype error,skype error,0
...,...,...,...
8495,emails not coming in from zz mail,\r\n\r\nreceived from: avglmrts.vhqmtiua@gmail...,29
8496,telephony_software issue,telephony_software issue,0
8497,vip2: windows password reset for tifpdchb pedx...,vip2: windows password reset for tifpdchb pedx...,0
8498,machine não está funcionando,i am unable to access the machine utilities to...,62


In [0]:
# Treating missing values
columns = ["Short description", "Description"]
df1[columns]=df1[columns].fillna(df1.mode().iloc[0])  

In [0]:

# Verifying the non null count post treating missing values
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8500 entries, 0 to 8499
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Short description  8500 non-null   object
 1   Description        8500 non-null   object
 2   Assignment group   8500 non-null   object
dtypes: object(3)
memory usage: 199.3+ KB


In [0]:
#Converting the dependent variable column to integer datatype for multiclass clasificatioon
df1['Assignment group'] = df1['Assignment group'].astype(int)

In [0]:
# Verifying the data set post converting the data type of dependent variable
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8500 entries, 0 to 8499
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Short description  8500 non-null   object
 1   Description        8500 non-null   object
 2   Assignment group   8500 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 199.3+ KB


In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [0]:
stop_words = set(stopwords.words('english')) 

def text_cleaner(text,num):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)

    if(num==0):
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens=newString.split()
    
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [0]:
#call the function
cleaned_Short_description = []
for t in df1['Short description']:
    cleaned_Short_description.append(text_cleaner(t,0)) 

In [0]:
cleaned_Short_description[:10]

['login issue',
 'outlook',
 'cant log vpn',
 'unable access hr tool page',
 'skype error',
 'unable log engineering tool skype',
 'event critical hostname company com value mountpoint threshold oracle sid erpdata',
 'ticket employment status new non employee enter user name',
 'unable disable add ins outlook',
 'ticket update inplant']

In [0]:
#call the function
cleaned_Decription = []
for t in df1['Description']:
    cleaned_Decription.append(text_cleaner(t,0)) 

In [0]:
cleaned_Decription[:10]

['verified user details checked user name ad reset password advised user login check caller confirmed able login issue resolved',
 'received hmjdrvpb komuaywn gmail com hello team meetings skype meetings etc appearing outlook calendar somebody please advise correct kind',
 'received eylqgodm ybqkwiam gmail com hi cannot log vpn best',
 'unable access hr tool page',
 'skype error',
 'unable log engineering tool skype',
 'event critical hostname company com value mountpoint threshold oracle sid erpdata sr psa sr psa data perpsr psa',
 'ticket employment status new non employee enter user name',
 'unable disable add ins outlook',
 'ticket update inplant']

In [0]:
df1['Short description']=cleaned_Short_description
df1['Description']=cleaned_Decription

In [0]:
df1.head()

,Short description,Description,Assignment group
0,login issue,verified user details checked user name ad reset password advised user login check caller confirmed able login issue resolved,0
1,outlook,received hmjdrvpb komuaywn gmail com hello team meetings skype meetings etc appearing outlook calendar somebody please advise correct kind,0
2,cant log vpn,received eylqgodm ybqkwiam gmail com hi cannot log vpn best,0
3,unable access hr tool page,unable access hr tool page,0
4,skype error,skype error,0


In [0]:
max_length = 0
max_len_line = ''
file = open("input_data.csv")
for line in file:
    if(len(line) > max_length):
        max_length = len(line)
        max_len_line = line
print(max_len_line)
print(max_length)

system.outofmemoryexception: exception of type 'system.outofmemoryexception' was thrown. at system.reflection.runtimeassembly.getrawbytes(runtimeassembly assembly, objecthandleonstack retrawbytes) at system.reflection.runtimeassembly.getrawbytes() at system.security.policy.hash.getrawdata() at system.security.policy.hash.generatehash(type hashtype) at system.security.policy.hash.get_sha1() at system.web.handlers.scriptresourcehandler.getassemblyinfointernal(assembly assembly) at system.web.handlers.scriptresourcehandler.getassemblyinfo(assembly assembly) at system.web.handlers.scriptresourcehandler.runtimescriptresourcehandler.getscriptresourceurlimpl(list`1 assemblyresourcelists, boolean zip) at system.web.handlers.scriptresourcehandler.runtimescriptresourcehandler.system.web.handlers.iscriptresourcehandler.getscriptresourceurl(list`1 assemblyresourcelists, boolean zip) at system.web.handlers.scriptresourcehandler.runtimescriptresourcehandler.system.web.handlers.iscriptresourcehandler

In [0]:
max_features = 10000
maxlen = 1795
embedding_size = 200

In [0]:
tokenizer = Tokenizer(num_words=max_features, split='""')
tokenizer.fit_on_texts(df1['Short description'].values)

In [0]:
tokenizer = Tokenizer(num_words=max_features, split='""')
tokenizer.fit_on_texts(df1['Description'].values)

In [0]:
X = tokenizer.texts_to_sequences(df1['Short description'])
X = tokenizer.texts_to_sequences(df1['Description'])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(df['Assignment group'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 8500
[0 0 0 ... 0 0 0]
Number of Labels:  8500
GRP_0


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

In [0]:
num_words = len(tokenizer.word_index) + 1